# Ouverture

Une idée de projet à faire avec l'aide de ce chiffrement serait de transférer des données chiffrées d'un appareil à un autre. La transmission entre les deux serait une transmition en série, c'est à dire que les appareils seraient connectés matériellement entre eux.
Pour exemple, nous pouvons voir ce genre de transmitions séries par des BBC Microbit. 
Pour cela, connectons ensemble deux microbit de la manière suivante (<a href="https://microbit-micropython.readthedocs.io/fr/latest/tutorials/network.html">source de l'image et du code ci-dessous</a>)
<img src="BBCMicroBit.png">
Ainsi nous pouvons voir clairement que les ports 2 des Microbit sont les sorties et les ports 1 sont les entrées.
Chaque port 1 est relié via deux pinces crocodiles et un câble à un port 2.
Nous allons donc avec, cela, faire un protocole de communication. Les ordinateurs doivent s’entendre avant de savoir comment envoyer et recevoir des messages. Le protocole le plus connu à ce jour est surement le protocole de transfert hypertexte (HTTP) utilisé par le World Wide Web (c'est les trois **w** que l'on retrouve souvent dans notre barre de recherche pour rentrer des liens).
Mais avant de divaguer sur d'autres sujets, rappellons qu'un protocole de communication très ancien nous permettait aussi de communiquer, c'est le code Morse (<a href="https://fr.wikipedia.org/wiki/Code_Morse_international">la p'tite page wikipédia)</a>.
Nous allons maintenant comment deux BBC Microbit vont communiquer en série entre elles par les explications suivantes !

#### Tout d'abord, il faut définir quels signaux corresponds à sa lettre ou à son chiffre

In [ ]:
# Une table de correspondance du code Morse. Un point correspond à un appui très bref, c'est à dire que le signal est petit
MORSE_CODE_LOOKUP = { #Et si le signal est un tiret, alors c'est un appui plus long que le point, c'est à dire que le signal est plus long
    ".-": "A",
    "-...": "B",
    "-.-.": "C",
    "-..": "D",
    ".": "E",
    "..-.": "F",
    "--.": "G",
    "....": "H",
    "..": "I",
    ".---": "J",
    "-.-": "K",
    ".-..": "L",
    "--": "M",
    "-.": "N",
    "---": "O",
    ".--.": "P",
    "--.-": "Q",
    ".-.": "R",
    "...": "S",
    "-": "T",
    "..-": "U",
    "...-": "V",
    ".--": "W",
    "-..-": "X",
    "-.--": "Y",
    "--..": "Z",
    ".----": "1",
    "..---": "2",
    "...--": "3",
    "....-": "4",
    ".....": "5",
    "-....": "6",
    "--...": "7",
    "---..": "8",
    "----.": "9",
    "-----": "0"
}

### C'est bien beau d'avoir des données, mais il faut pouvoir les stockers et ainsi comprendre un message, c'est pour cela que le **buffer**  <a href="http://fr.lmgtfy.com/?q=Qu%27est+ce+qu%27un+buffer+en+informatique">(définition buffer)</a>

In [ ]:
from microbit import * #Le bip sonore de la BBC Microbit
import music

def decode(buffer): #Nous créons une fonction
    # Essaie d'obtenir une correspondance entre le contenu du buffer et le
    # code Morse. Si il n'y en a pas, renvoie juste un point.
    return CODE_MORSE_CORRESPONDANCE.get(buffer, '.')

### Définissons ensuite le point ainsi que le tiret et comment ils vont être créés

In [ ]:
# Image correspondant à un POINT.
POINT = Image("00000:"
            "00000:"
            "00900:"
            "00000:"
            "00000:")


# Image correspondant à un TIRET.
TIRET = Image("00000:"
             "00000:"
             "09990:"
             "00000:"
             "00000:")


# Pour créer un POINT tu dois maintenir le bouton pendant moins de 250ms, le temps est très souvent exprimé en millisecondes
SEUIL_POINT = 250
# Pour créer un TIRET tu dois maintenir le bouton pendant moins de 500ms
SEUIL_TIRET = 500

Ensuite il faut définir le chemin que doit faire l'information pour être stocké par le buffer, ainsi que le code Morse traduit. Sans oublier de vérifier depuis combien de temps le bouton est préssé.

In [ ]:
# Contient le signal Morse entrant.
buffer = ''
# Contient le Morse traduit en caractères.
message = ''
# Le temps depuis lequel l'appareil a attendu le prochain appui sur une touche.
debut_attente = running_time()

Vient maintenant la partie du protocole du programme, pour le bon fonctionnement de cette transmition série.

In [ ]:
# Met l'appareil dans une boucle pour attendre et réagir aux appuis sur un
# bouton
while True:
    # Détermine le temps que l'appareil a attendu l'appui
    attente = running_time() - debut_attente
    # Réinitialise l'horodatage de temp_presse
    temps_touche_presse = None
    # Si le bouton A est mainteu appuyé alors...
    while button_a.is_pressed():
        # Emet un bip - c'est du code Morse tu sais ;-)
        music.pitch(880, 10)
        # Met le pin1 (sortie) sur "on"
        pin1.write_digital(1)
        # ...et si il n'y a pas encore de temps_touche_presse alors on le met maintenant!
        if not temps_touche_presse:
            temps_touche_presse = running_time()
    # Alternativment, si le pin2 (input) reçoit un signal, on fait comme  si
    # c'était un appui sur le bouton A
    while pin2.read_digital():
        if not temps_touche_presse:
            temps_touche_presse = running_time()
    # On récupère l'heure actuelle et on l'appelle temps_touche_haute
    temps_touche_haute = running_time()
    # Met le pin1 (sortie) sur "off"
    pin1.write_digital(0)
    # Si il y a un temps_touche_presse (créé lorsque le bouton A a été pressé
    # la  première fois).
    if temps_touche_presse:
        # ...détermine depuis combien de temps il est appuyé
        duree = temps_touche_haute - temps_touche_presse
        # Si la durée est inférieur à la durée maximale d'un DOT...
        if duree < SEUIL_POINT:
            # ... alors ajoute un point au buffer contenant le code Morse entrant
            # et montre un point sur l'affichage..
            buffer += '.'
            display.show(POINT)
        # Sinon, si la durée  est inférieur à la durée maximale d'un TIRET...
        # (mais plus longue que celle d'un POINT ~qui a été géré avant)
        elif duree < SEUIL_TIRET:
            # ... alors ajoute un tiret au buffer contenant le code Morse entrant
            # et affiche-le.
            buffer += '-'
            display.show(TIRET)
        # Sinon, toutes les autres durée d'appui sont ignorées (ce n'est pass
        # nécessaire mais on le met pour la compréhension)
        else:
            pass
        # L'appui sur le bouton a été géré, on peut réinitialiser le temps
        # d'attente pour le prochain appui.
        debut_attente = running_time()

    # Sinon, il n'y a pas eu d'appui pendant ce cycle de la boucle, donc il
    # faut vérifier qu'il n'y a pas eu de pause indiquant la fin du
    # code Morse du caractère entrant. La pause doit être plus longue que
    # la durée du code d'un TIRET.
    elif len(buffer) > 0 and attente > SEUIL_TIRET:
        # Le buffer n'est pas vide et on a atteitn la fin du code donc...
        # il faut décoder le buffer entrant.
        character = decode(buffer)
        # Puis réinitialiser le buffer
        buffer = ''
        # Afficher le caractère décodé
        display.show(character)
        # et ajouter le caractère au message.
        message += character
    # Enfin, si le bouton B a été appuyé pendant tout ce temps là...
    if button_b.was_pressed():
        # ... affiche le message,
        display.scroll(message)
        # et réinitialise-le  (prêt pour un nuoveau message).
        message = ''

### Vient donc ensuite le code en entier qui devrait être compréhensible avec les explications précédentes !

In [ ]:
from microbit import * #Le bip sonore de la BBC Microbit 
import music

# Une table de correspondance du code Morse.
MORSE_CODE_LOOKUP = {
    ".-": "A",
    "-...": "B",
    "-.-.": "C",
    "-..": "D",
    ".": "E",
    "..-.": "F",
    "--.": "G",
    "....": "H",
    "..": "I",
    ".---": "J",
    "-.-": "K",
    ".-..": "L",
    "--": "M",
    "-.": "N",
    "---": "O",
    ".--.": "P",
    "--.-": "Q",
    ".-.": "R",
    "...": "S",
    "-": "T",
    "..-": "U",
    "...-": "V",
    ".--": "W",
    "-..-": "X",
    "-.--": "Y",
    "--..": "Z",
    ".----": "1",
    "..---": "2",
    "...--": "3",
    "....-": "4",
    ".....": "5",
    "-....": "6",
    "--...": "7",
    "---..": "8",
    "----.": "9",
    "-----": "0"
}


def decode(buffer):
    # Essaie d'obtenir une correspondance entre le contenu du buffer et le
    # code Morse. Si il n'y en a pas, renvoie juste un point.
    return CODE_MORSE_CORRESPONDANCE.get(buffer, '.')


# Image correspondant à un POINT.
POINT = Image("00000:"
            "00000:"
            "00900:"
            "00000:"
            "00000:")


# Image correspondant à un TIRET.
TIRET = Image("00000:"
             "00000:"
             "09990:"
             "00000:"
             "00000:")


# Pour créer un POINT tu dois maintenir le bouton pendant moins de 250ms
SEUIL_POINT = 250
# Pour créer un TIRET tu dois maintenir le bouton pendant moins de 500ms
SEUIL_TIRET = 500


# Contient le signal Morse entrant.
buffer = ''
# Contient le Morse traduit en caractères.
message = ''
# Le temps depuis lequel l'appareil a attendu le prochain appui sur une touche.
debut_attente = running_time()


# Met l'appareil dans une boucle pour attendre et réagir aux appuis sur un
# bouton
while True:
    # Détermine le temps que l'appareil a attendu l'appui
    attente = running_time() - debut_attente
    # Réinitialise l'horodatage de temp_presse
    temps_touche_presse = None
    # Si le bouton A est mainteu appuyé alors...
    while button_a.is_pressed():
        # Emet un bip - c'est du code Morse tu sais ;-)
        music.pitch(880, 10)
        # Met le pin1 (sortie) sur "on"
        pin1.write_digital(1)
        # ...et si il n'y a pas encore de temps_touche_presse alors on le met maintenant!
        if not temps_touche_presse:
            temps_touche_presse = running_time()
    # Alternativment, si le pin2 (input) reçoit un signal, on fait comme  si
    # c'était un appui sur le bouton A
    while pin2.read_digital():
        if not temps_touche_presse:
            temps_touche_presse = running_time()
    # On récupère l'heure actuelle et on l'appelle temps_touche_haute
    temps_touche_haute = running_time()
    # Met le pin1 (sortie) sur "off"
    pin1.write_digital(0)
    # Si il y a un temps_touche_presse (créé lorsque le bouton A a été pressé
    # la  première fois).
    if temps_touche_presse:
        # ...détermine depuis combien de temps il est appuyé
        duree = temps_touche_haute - temps_touche_presse
        # Si la durée est inférieur à la durée maximale d'un DOT...
        if duree < SEUIL_POINT:
            # ... alors ajoute un point au buffer contenant le code Morse entrant
            # et montre un point sur l'affichage..
            buffer += '.'
            display.show(POINT)
        # Sinon, si la durée  est inférieur à la durée maximale d'un TIRET...
        # (mais plus longue que celle d'un POINT ~qui a été géré avant)
        elif duree < SEUIL_TIRET:
            # ... alors ajoute un tiret au buffer contenant le code Morse entrant
            # et affiche-le.
            buffer += '-'
            display.show(TIRET)
        # Sinon, toutes les autres durée d'appui sont ignorées (ce n'est pass
        # nécessaire mais on le met pour la compréhension)
        else:
            pass
        # L'appui sur le bouton a été géré, on peut réinitialiser le temps
        # d'attente pour le prochain appui.
        debut_attente = running_time()

    # Sinon, il n'y a pas eu d'appui pendant ce cycle de la boucle, donc il
    # faut vérifier qu'il n'y a pas eu de pause indiquant la fin du
    # code Morse du caractère entrant. La pause doit être plus longue que
    # la durée du code d'un TIRET.
    elif len(buffer) > 0 and attente > SEUIL_TIRET:
        # Le buffer n'est pas vide et on a atteitn la fin du code donc...
        # il faut décoder le buffer entrant.
        character = decode(buffer)
        # Puis réinitialiser le buffer
        buffer = ''
        # Afficher le caractère décodé
        display.show(character)
        # et ajouter le caractère au message.
        message += character
    # Enfin, si le bouton B a été appuyé pendant tout ce temps là...
    if button_b.was_pressed():
        # ... affiche le message,
        display.scroll(message)
        # et réinitialise-le  (prêt pour un nuoveau message).
        message = ''

Pour faciliter la programmation pour effectuer des communications séries il existe une library UART.
Mais avant cela, l'UART (Universal Asynchronous Receiver Transmitter) est, en langage courant, le composant utilisé pour faire la liaison entre l'ordinateur et le port série. Plus scientifiquement c'est un émetteur-récepteur asynchrone universel.
Ce composant et cette library vont nous êtres d'une grande utilité pour établir des liaisons série et effectuer des communications séries entre deux joueurs par exemple. Le code ci-dessous vous permettra d'établir ce genre de connexion par exemple.
<a href="http://www.multiwingspan.co.uk/micro.php?page=pybit2bit">Source du code ci-dessous</a>

In [ ]:
from microbit import *

sleep(5000) #Repos pour 5 secondes
uart.init(baudrate=9600, bits=8, parity=None, stop=1, tx=pin0, rx=pin1) #baudrate est la vitesse de bit par seconde

def Send(m): #Nous définissons une fonction 
    b = [] #Création d'un tableau
    for c in m: #Boucle for 
        b.append(ord(c))
    d = bytes(b)
    uart.write(d) #Nous utilisons la library UART

def Receive(): #Nous définissons la fonction Receive
    if uart.any(): #Nous utilisons la library UART
        m = uart.readall() #Nous utilisons la library UART et nous initialisons et déclarons une variable
        s = "" #Initialisation et déclaration d'une variable.
        for b in m:
            s = s + chr(b)
        display.scroll(s)
        
while True: #Boucle tant que c'est vrai
    Receive() #Exécuter la fonction Receive
    if button_a.is_pressed(): #Si le bouton a est préssé 
        Send("HELLO") #Alors la fonction Send s'exécute et envoie le mot HELLO
        sleep(500) #Repos pendant 0.5 seconde
    sleep(20)

 Il existe aussi des block avancés pour effectuer une communication entre deux Microbit, qui peuvent être utile pour diffuser l'information sur les leds de la Microbit. Pour exemple, si on prend la commande suivante : print("this") et bien cette information sera qu'un commentaire pour la Microbit qui reçoit l'information. <img src="BlockAvance.PNG"><a href="https://makecode.microbit.org/reference/serial">(Block avancés pour BBC Microbit)</a>